# Head

In [ ]:
%pip install mambapy

In [1]:
import torch, transformers, importlib, InitMamba
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer

# Lab

In [9]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
inputs = tokenizer("Today is Sunday.", return_tensors="pt", max_length=10, padding='max_length')
inputs

{'input_ids': tensor([[14569,   310,  6926,    15,     0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])}

In [10]:
importlib.reload(InitMamba)
import InitMamba

config = MambaConfig.from_pretrained("state-spaces/mamba-130m-hf")
config.use_mambapy = True
model1 = InitMamba.MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf", config=config) #.to('cuda')
# model1.train()

res1 = model1(**inputs, output_ssm_last_states = True)
ssm_last_states = res1.ssm_last_states

In [11]:
# Model without initial SSM states gives random results
print(tokenizer.batch_decode(res1.logits.argmax(dim=-1)))

# Model with initial SSM states gives "Sunday"
res2 = model1(**inputs, inputs_ssm_states = ssm_last_states)
print(tokenizer.batch_decode(res2.logits.argmax(dim=-1)))

[', the,\n<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>']
[' is Sunday,\n<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>']


# Check same

In [12]:
model2 = MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf", config=config)

model1.eval()
model2.eval()

print((model2(**inputs).logits - model1(**inputs).logits).abs().sum())

tensor(0., grad_fn=<SumBackward0>)
